# Step 1: Set up the LLM app

We're using a [Q&A example from LangChain](https://python.langchain.com/docs/use_cases/question_answering/quickstart) where we'll be doing a Q&A over a [blog](https://lilianweng.github.io/posts/2023-06-23-agent/).

In this notebook we setup an LLM and a RAG system via langchain. This is the "app" that we will be evaluating in the next few notebooks.

NOTE: This notebook and the following notebooks use OpenAI models to run code. You can set your API key in the `.env` file located in the root directory of this workshop's folder. We estimate that running all the notebooks together will cost $25 with OpenAI's models.

In [1]:
%reload_ext autoreload
%autoreload 2

import dotenv
dotenv.load_dotenv("../.env", override=True)

True

In [2]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [3]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


In [4]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [5]:
# Setup LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

## You can replace this with an LLM of your choice or your fine-tuned LLM
## Refer: https://python.langchain.com/docs/integrations/llms/
# llm = HuggingFacePipeline.from_model_id(
#     model_id="microsoft/Phi-3-mini-4k-instruct",
#     task="text-generation",
#     pipeline_kwargs={"max_new_tokens": 100, "temperature": 0},
#)

def format_to_string_list(docs):
    return [doc.page_content for doc in docs]

def concat_string(str_list):
    return "\n\n".join(str_list)


retriever_block = retriever | format_to_string_list
generative_block = prompt | llm | StrOutputParser()


In [6]:
question = "What is Task Decomposition?"
context_list = retriever_block.invoke("What is Task Decomposition?")
print("Context:", context_list)
print("\n")

answer = generative_block.invoke({"question":question, "context":concat_string(context_list)})
print("Answer:", answer)

Context: ['Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.', 'Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) o